In [ ]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import random
import rawpy
from PIL import Image

In [ ]:
def load_calibration(calib_file):
    """Loads stereo calibration parameters from an .npz file."""
    with np.load(calib_file) as data:
        map1x = data['map1x']
        map1y = data['map1y']
        map2x = data['map2x']
        map2y = data['map2y']
        Q = data['Q']
        T = data['T']
        R = data['R']
    
    return map1x, map1y, map2x, map2y, Q, T, R

def rectify_stereo_pair(img0, img1, map1x, map1y, map2x, map2y):
    """
    Takes a new stereo pair (img0, img1), rectifies them using the provided maps,
    and returns the undistorted, rectified pair.
    """
    rect0 = cv2.remap(img0, map1x, map1y, cv2.INTER_LINEAR)
    rect1 = cv2.remap(img1, map2x, map2y, cv2.INTER_LINEAR)
    return rect0, rect1

In [ ]:
## RAW Images


map1x, map1y, map2x, map2y, Q, T, R = load_calibration("calib_data1.npz")

directory = "./calib_data1"
ir_raws = glob.glob(os.path.join(directory, "*.dng"))
rgb_raws = glob.glob(os.path.join(directory, "*.dng"))
ir_raws.sort()
rgb_raws.sort()

rand_indx = random.randint(0, len(rgb_raws)-1)
print(f"Choosing images: {ir_raws[rand_indx]} and {rgb_raws[rand_indx]}")

with rawpy.imread(ir_raws[rand_indx]) as raw0:
    # raw0.raw_image is the un-demosaiced, black-level-subtracted array
    img0 = raw0.raw_image.copy()
with rawpy.imread(rgb_raws[rand_indx]) as raw1:
    img1 = raw1.raw_image.copy()

# img0 = cv2.imread(ir_images[rand_indx])
# img1 = cv2.imread(rgb_images[rand_indx])

rectified0, rectified1 = rectify_stereo_pair(img0, img1, map1x, map1y, map2x, map2y)

Image.fromarray(rectified0).save("rectified_ir.tiff")
Image.fromarray(rectified1).save("rectified_rgb.tiff")

# cv2.imwrite("rectified_ir.jpg", rectified0)
# cv2.imwrite("rectified_rgb.jpg", rectified1)

In [ ]:
#color_0 = cv2.cvtColor(rectified1, cv2.COLOR_BAYER_RG2BGR)
#rectified1_8bit = cv2.convertScaleAbs(rectified1, alpha=(255.0/65535.0))  # Convert 16-bit to 8-bit
color_0 = cv2.cvtColor(rectified1, cv2.COLOR_BAYER_RG2BGR)  # Demosaic the raw array
color_0 = cv2.convertScaleAbs(color_0, alpha=(255.0/65535.0)) 
plt.imshow(cv2.cvtColor(color_0, cv2.COLOR_BGR2RGB))
avg_values = color_0.mean(axis=(0, 1))
print(f"Average values of each channel: {avg_values}")
plt.title("Demosaiced Image")
plt.axis("off")
plt.title("Demosaiced Image (BGR)")
plt.axis("off")
plt.show()
# print(rectified1_8bit.shape)
# plt.imshow(rectified1_8bit)
# plt.show

In [ ]:
map1x, map1y, map2x, map2y, Q, T, R = load_calibration("two_rgb.npz")

ir_images = [] # Right camera
rgb_images = [] # Left camera
directory = "./two_rgb"
for filename in os.listdir(directory):
    if filename.endswith("_ir.jpg"):
        ir_images.append(os.path.join(directory, filename))
    elif filename.endswith("_rgb.jpg"):
        rgb_images.append(os.path.join(directory, filename))
ir_images.sort()
rgb_images.sort()
print("IR images: ", ir_images)
print("RGB images: ", rgb_images)

In [ ]:
rand_indx = random.randint(0, len(ir_images)-1)
print(f"Choosing images: {ir_images[rand_indx]} and {rgb_images[rand_indx]}")
img0 = cv2.imread(ir_images[rand_indx])
img1 = cv2.imread(rgb_images[rand_indx])

rectified0, rectified1 = rectify_stereo_pair(img0, img1, map1x, map1y, map2x, map2y)

cv2.imwrite("rectified_ir.jpg", rectified0)
cv2.imwrite("rectified_rgb.jpg", rectified1)

left_img = rectified0
right_img = rectified1

# Convert to grayscale for easier visual comparison
# gray_left = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
# gray_right = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
gray_left = left_img[:, :, 2]
gray_right = right_img[:, :, 2]

s = ssim(gray_left, gray_right)
print(s)

# Display images side by side
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(gray_left, cmap="gray")
plt.title("Rectified Left Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(gray_right, cmap="gray")
plt.title("Rectified Right Image")
plt.axis("off")

plt.show()

In [ ]:
def draw_epipolar_lines(img1, img2, num_lines=10):
    h, w = img1.shape[:2]
    step = h // num_lines  # Spacing between lines

    img1_lines = img1.copy()
    img2_lines = img2.copy()

    for i in range(0, h, step):
        cv2.line(img1_lines, (0, i), (w, i), (0, 255, 0), 1)
        cv2.line(img2_lines, (0, i), (w, i), (0, 255, 0), 1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img1_lines, cv2.COLOR_BGR2RGB))
    plt.title("Left Image with Epipolar Lines")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(img2_lines, cv2.COLOR_BGR2RGB))
    plt.title("Right Image with Epipolar Lines")
    plt.axis("off")

    plt.show()

# Load rectified images

draw_epipolar_lines(gray_left, gray_right)


In [ ]:
# stereo = cv2.StereoSGBM_create(
#     minDisparity=0, numDisparities=64, blockSize=11, P1=8*3*11**2, P2=32*3*11**2, disp12MaxDiff=1, uniquenessRatio=10, speckleWindowSize=100, speckleRange=32
# )
# disparity_raw = stereo.compute(gray_left, gray_right)  # This is int16
# disparity = disparity_raw.astype(np.float32) / 16.0

win_size = 5
min_disp = 0
max_disp = 128
num_disp = max_disp - min_disp   # Needs to be divisible by 16

# Update StereoSGBM_create parameters
stereo = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=win_size,
    uniquenessRatio=15,
    speckleWindowSize=100,
    speckleRange=2,
    disp12MaxDiff=1,
    P1=8 * win_size**2,
    P2=128 * win_size**2,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)

# blur_ksize = (5, 5) # Experiment with kernel size
# gray_left_blurred = cv2.GaussianBlur(gray_left, blur_ksize, 0)
# gray_right_blurred = cv2.GaussianBlur(gray_right, blur_ksize, 0)

# disparity = stereo.compute(gray_left_blurred, gray_right_blurred).astype(np.float32) / 16.0
disparity = stereo.compute(gray_left, gray_right).astype(np.float32) / 16.0

plt.figure(figsize=(10, 6))
plt.imshow(disparity, cmap='jet')
plt.colorbar()
plt.title("Disparity Map")
plt.axis("off")
plt.show()

In [ ]:
def compute_filtered_disparity(left_img, right_img):
    # Convert to grayscale if needed
    if len(left_img.shape) == 3:
        gray_left = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    else:
        gray_left = left_img
    if len(right_img.shape) == 3:
        gray_right = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
    else:
        gray_right = right_img

    # Stereo matcher parameters
    win_size = 5
    min_disp = 0
    num_disp = 128  # Must be divisible by 16

    left_matcher = cv2.StereoSGBM_create(
        minDisparity=min_disp,
        numDisparities=num_disp,
        blockSize=win_size,
        P1=8 * win_size ** 2,
        P2=32 * win_size ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=200,
        speckleRange=2,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )

    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

    # Compute both disparities
    disp_left = left_matcher.compute(gray_left, gray_right).astype(np.int16)
    disp_right = right_matcher.compute(gray_right, gray_left).astype(np.int16)

    # WLS filter
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
    wls_filter.setLambda(8000)
    wls_filter.setSigmaColor(1.5)

    filtered_disp = wls_filter.filter(disp_left, gray_left, None, disp_right)
    filtered_disp = filtered_disp.astype(np.float32) / 16.0

    # Optional: normalize for visualization
    disp_vis = cv2.normalize(filtered_disp, None, alpha=0, beta=255,
                             norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return filtered_disp, disp_vis

def warp_right_to_left(right_img, disparity_map):
    """Warp right image using the filtered disparity map to align with left image"""
    h, w = disparity_map.shape
    map_x, map_y = np.meshgrid(np.arange(w), np.arange(h))
    map_x = map_x.astype(np.float32)
    map_y = map_y.astype(np.float32)

    # x_right = x_left - disparity
    map_x_warp = map_x - disparity_map

    warped = cv2.remap(right_img, map_x_warp, map_y, cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    return warped

def overlay_images(left_img, warped_right):
    """Overlay left and warped right image for visualization"""
    overlay = cv2.addWeighted(left_img, 0.5, warped_right, 0.5, 0)
    return overlay

In [ ]:
disp, vis_disp = compute_filtered_disparity(rectified0, rectified1)
warped_right = warp_right_to_left(rectified1, disp)
overlay = overlay_images(cv2.cvtColor(rectified0, cv2.COLOR_BGR2GRAY),
                         cv2.cvtColor(warped_right, cv2.COLOR_BGR2GRAY))
# === Show everything ===
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(disp, cmap='plasma')
plt.title("Filtered Disparity")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(warped_right, cv2.COLOR_BGR2RGB))
plt.title("Warped Right → Left")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(overlay, cmap='gray')
plt.title("Overlay (L + Warped R)")
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
h, w = disparity.shape
map_x = np.tile(np.arange(w), (h, 1)).astype(np.float32)
map_y = np.tile(np.arange(h).reshape(-1, 1), (1, w)).astype(np.float32)

# Subtract disparity from NIR x-coordinates to align it to RGB
right_warped_to_left = cv2.remap(gray_right, map_x - disparity, map_y,
                        interpolation=cv2.INTER_LINEAR,
                        borderMode=cv2.BORDER_CONSTANT,
                        borderValue=0)
plt.imshow(right_warped_to_left, cmap='gray')
plt.show()
overlay = cv2.addWeighted(gray_left, 0.5, right_warped_to_left, 0.5, 0)
plt.imshow(overlay)
plt.show()

In [ ]:
# --- Assume gray_left and gray_right are loaded and are 8-bit grayscale ---
# Example:
# img_left_color = cv2.imread('path/to/left_image.png')
# img_right_color = cv2.imread('path/to/right_image.png')
# gray_left = cv2.cvtColor(img_left_color, cv2.COLOR_BGR2GRAY)
# gray_right = cv2.cvtColor(img_right_color, cv2.COLOR_BGR2GRAY)

# --- 1. Define SGBM Parameters (Use your tuned values) ---
win_size = 9 # Example: smaller block size
min_disp = 0 # Example: start from zero
num_disp = 128 # Example: must be > 0 and divisible by 16

# Ensure num_disp is valid based on min_disp and desired max_disp
# max_disp = min_disp + num_disp

# Example P1, P2 for grayscale (remove * 3 if inputs are grayscale)
win_size = 9
min_disp = 32
max_disp = 256
num_disp = max_disp - min_disp   # Needs to be divisible by 16

# Update StereoSGBM_create parameters
left_matcher = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=win_size,
    uniquenessRatio=15,
    speckleWindowSize=100,
    speckleRange=2,
    disp12MaxDiff=1,
    P1=8 * win_size**2,
    P2=128 * win_size**2,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)

# --- (Optional) Pre-processing ---
# blur_ksize = (5, 5)
# gray_left_blurred = cv2.GaussianBlur(gray_left, blur_ksize, 0)
# gray_right_blurred = cv2.GaussianBlur(gray_right, blur_ksize, 0)
# Use blurred images below if you apply this step

# --- 2. Create Right Matcher and Compute Right Disparity ---
right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

# Compute raw disparity map (CV_16S format)
# Input order is important: reference=right, matching=left
disp_right_raw = right_matcher.compute(gray_right, gray_left)
print(f"Raw Right Disparity Stats:")
print(f"  dtype: {disp_right_raw.dtype}")
print(f"  min: {np.min(disp_right_raw)}")
print(f"  max: {np.max(disp_right_raw)}")

# Convert to float and scale (disparity values are scaled by 16)
disparity_R = disp_right_raw.astype(np.float32) / 16.0
print(f"\nScaled Right Disparity Stats:")
print(f"  dtype: {disparity_R.dtype}")
print(f"  min: {np.min(disparity_R)}")
print(f"  max: {np.max(disparity_R)}")

# --- (Optional) Apply WLS Filter for smoother disparity ---
# If you use WLS, it's often applied to the *left* disparity,
# and the right disparity is used internally by the filter.
# You could potentially filter disp_right_raw too, but let's skip for now.

# --- 3. Create Remapping Grids ---
h, w = gray_right.shape[:2] # Use dimensions of the *target* image (right)
print(h, w)

# Base grids corresponding to (x_r, y_r) coordinates
map_x_right_base = np.tile(np.arange(w), (h, 1)).astype(np.float32)
map_y_right_base = np.tile(np.arange(h).reshape(-1, 1), (1, w)).astype(np.float32)

# --- 4. Calculate Source Coordinates in Left Image ---
# map1[y_r, x_r] = x_l = x_r + disparity_R[y_r, x_r]
map_x_source = map_x_right_base - disparity_R

# map2[y_r, x_r] = y_l = y_r
map_y_source = map_y_right_base

################
# Compute the left disparity the standard way:
disp_left_raw = left_matcher.compute(gray_left, gray_right)
disparity_L = disp_left_raw.astype(np.float32) / 16.0

# Then warp with x_left = x_right + disparity_L
map_x_source = map_x_right_base + disparity_L
map_y_source = map_y_right_base
###############


# --- 5. Perform Remapping ---
# Warp gray_left using the calculated maps
left_warped = cv2.remap(
    gray_left,          # Source image to warp
    map_x_source,       # x-coordinates in gray_left to sample from
    map_y_source,       # y-coordinates in gray_left to sample from
    interpolation=cv2.INTER_LINEAR,
    borderMode=cv2.BORDER_CONSTANT, # Handle pixels mapped outside source
    borderValue=0                   # Value for pixels outside source
)

# --- 6. Visualization ---
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(disparity_L, cmap='jet')
plt.colorbar()
plt.title("Right Disparity Map (R -> L)")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(left_warped, cmap='gray')
plt.title("Warped Left Image (Aligned to Right)")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(gray_right, cmap='gray')
plt.title("Reference Image (Right)")
plt.axis("off")

plt.tight_layout()
plt.show()

# Optional: Overlay to check alignment
overlay = cv2.addWeighted(cv2.cvtColor(left_warped, cv2.COLOR_GRAY2BGR), 0.5,
                          cv2.cvtColor(gray_right, cv2.COLOR_GRAY2BGR), 0.5, 0)
plt.figure()
plt.imshow(overlay)
plt.title("Overlay (Warped Left + Right)")
plt.show()

In [ ]:

def warp_right_to_left(right_img, disparity_map):
    """
    Warp the right image into the left camera's viewpoint using the disparity map.

    Parameters:
        right_img (H,W,3): The right camera rectified image.
        disparity_map (H,W): Disparity map in float32, representing shifts from left to right.

    Returns:
        warped_right_to_left (H,W,3): The warped right image in the left camera's frame.
    """
    h, w = disparity_map.shape
    # Generate a grid of pixel coordinates
    x_coords, y_coords = np.meshgrid(np.arange(w), np.arange(h))

    # To map right-to-left, shift each pixel in the right image by disparity pixels to the left
    # So the right image will be sampled at positions: x' = x - disparity
    map_x = (x_coords + disparity_map).astype(np.float32)
    map_y = y_coords.astype(np.float32)

    warped = cv2.remap(
        right_img, map_x, map_y,
        interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=(0, 0, 0)
    )
    return warped


In [ ]:

warped_right = warp_right_to_left(gray_left, disparity)
print(disparity.shape)
print(gray_left.shape)

plt.imshow(warped_right)

In [ ]:
print(disparity.shape)

In [ ]:
stacked = np.zeros((1520, 2028, 3), dtype=np.uint8)
stacked[:, :, 2] = warped_right
stacked[:, :, 0] = gray_right
print(stacked.shape)
plt.imshow(stacked, cmap='gray')

In [ ]:
stacked = np.zeros((3040, 4056, 3), dtype=np.uint8)
stacked[:, :, 2] = gray_left
stacked[:, :, 0] = gray_right
print(stacked.shape)
plt.imshow(stacked, cmap='gray')

In [ ]:
print(T)

In [ ]:
pandora = cv2.imread("./calib_data1/20250401_182846_ir.jpg")
pandora_bw = pandora[:, :, 2]
plt.imshow(pandora_bw, cmap='gray')